In [ ]:
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
import os
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from tqdm import tqdm
sns.set_palette('Dark2')
sns.set_context('paper')
sns.set_style({'axes.axisbelow': True, 
               'axes.edgecolor': '.15',
               'axes.facecolor': 'white',
               'axes.grid': True, 
               'axes.labelcolor': '.15', 
               'figure.facecolor': 'white', 
               'grid.color': '.15',
               'grid.linestyle': ':', 
               'grid.alpha': .5, 
               'image.cmap': 'Greys', 
               'legend.frameon': False, 
               'legend.numpoints': 1, 
               'legend.scatterpoints': 1,
               'lines.solid_capstyle': 'butt', 
               'axes.spines.right': False, 
               'axes.spines.top': False,  
               'text.color': '.15',  
               'xtick.top': False, 
               'ytick.right': False, 
               'xtick.color': '.15',
               'xtick.direction': 'out', 
               'ytick.color': '.15', 
               'ytick.direction': 'out', 
              })


import matplotlib

FONT_SIZE_PT = 5
matplotlib.rcParams['font.family'] = 'Arial'
matplotlib.rcParams['font.size'] = FONT_SIZE_PT
matplotlib.rcParams['axes.labelsize'] = FONT_SIZE_PT
matplotlib.rcParams['axes.titlesize'] = FONT_SIZE_PT
matplotlib.rcParams['figure.titlesize'] = FONT_SIZE_PT
matplotlib.rcParams['xtick.labelsize'] = FONT_SIZE_PT
matplotlib.rcParams['ytick.labelsize'] = FONT_SIZE_PT
matplotlib.rcParams['legend.fontsize'] = FONT_SIZE_PT
matplotlib.rcParams['legend.title_fontsize'] = FONT_SIZE_PT

matplotlib.rcParams['xtick.major.size'] = matplotlib.rcParams['ytick.major.size'] = 2
matplotlib.rcParams['xtick.major.width'] = matplotlib.rcParams['ytick.major.width'] = 0.5


matplotlib.rcParams['xtick.minor.size'] = matplotlib.rcParams['ytick.minor.size'] = 1

matplotlib.rcParams['xtick.minor.width'] = matplotlib.rcParams['ytick.minor.width'] = 0.5

matplotlib.rcParams['axes.linewidth'] = 0.5
matplotlib.rcParams['lines.linewidth'] = 0.5
matplotlib.rcParams['grid.linewidth'] = 0.25
matplotlib.rcParams['patch.linewidth'] = 0.25
matplotlib.rcParams['lines.markeredgewidth'] = 0.25
matplotlib.rcParams['lines.markersize'] = 2

FIVE_MM_IN_INCH = 0.19685
DPI = 600
matplotlib.rcParams['figure.figsize'] = (10 * FIVE_MM_IN_INCH, 9 * FIVE_MM_IN_INCH)
matplotlib.rcParams['savefig.dpi'] = DPI
matplotlib.rcParams['figure.dpi'] = DPI // 4


#http://phyletica.org/matplotlib-fonts/
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

# (03) Transformation and modeling

This notebook handles the core of the analysis:

1. Transformation of the numeric data into format that can be read by the model
2. Normalisation of the data
3. Modelling of differential enrichments

## Configuration

Input file (the numeric data from previous notebook):

In [ ]:
import pathlib
INPUT_NUMERIC_DATA = pathlib.Path('outputs') / '01-extracting' / 'data_numeric.csv'
assert INPUT_NUMERIC_DATA.is_file()

INPUT_METADATA = pathlib.Path('outputs') / '01-extracting' / 'data_metadata.csv'
assert INPUT_METADATA.is_file()


Output directory:

In [ ]:
import pathlib
OUTPUT_DIRECTORY = pathlib.Path('outputs') / '03-transformation-and-modelling'

if not OUTPUT_DIRECTORY.is_dir():
    OUTPUT_DIRECTORY.mkdir(parents=True)

Parameters, constants:

In [ ]:
# To do analysis, we will require that:
#- at least this many (below) values per condition are set (for the condition analysed)
MINIMUM_NUMBER_OF_OBSERVED_VALUES_PER_CONDITION = 2

# Cutoff for BH-adjusted p-values
FDR_THRESHOLD = 0.05

## Reading Metadata

Reading the metadata from previous notebook

In [ ]:
data_metadata = pd.read_csv(
    INPUT_METADATA,
    index_col=0,
)
data_metadata

## Reading numeric data

Read the numeric data from previous notebook:

In [ ]:
data_numeric = pd.read_csv(
    INPUT_NUMERIC_DATA,
    index_col=0
)
data_numeric.columns.name = 'Experiment_Replicate'
data_numeric

For this notebook it will also help to establish a set of "Headers" which describe what each column means:

In [ ]:
headers = []
for col in data_numeric.columns:
    headers.append([col] + list(col.split('_')))
    
headers = pd.DataFrame(headers, columns=[data_numeric.columns.name, 'Experiment', 'Replicate']).set_index('Experiment_Replicate')
headers

In [ ]:
data_numeric.describe()

The data we are loading here is MS intensities; they generally make more sense in log scale. 
We will therefore transform the data to `log2` here and work with that from now on:

In [ ]:
data_numeric_log2 = data_numeric.apply(np.log2)

## EDA (log2 transformed, unnormalised, unfiltered)

### Missing values

The first thing that is very obvious is that this data has many missing values:

In [ ]:
import missingno as msgno
msgno.matrix(data_numeric_log2, figsize=(20*FIVE_MM_IN_INCH, 20*FIVE_MM_IN_INCH), fontsize=6)

plt.title("Distribution of missing values in log2-transformed, unnormalised, and unfiltered data")
plt.xlabel("Data columns (experiments)")
plt.ylabel("Data rows (proteins)")
plt.xticks(rotation=90)

_fname = OUTPUT_DIRECTORY / '01-EDA-log2-transformed-unnormalised-unfiltered-missing-values.png'
_caption = """
Distribution of missing values in the log2-transformed, unnormalised, and unfiltered data. The heatmap above displays all 1046 proteins in the data in rows, and all experiments in columns. Dark values indicate the presence of a value, while white space indicates the absence of one. The sparkline on the right hand side counts the number of values per protein, which ranges from zero to 12 in this dataset.
"""
plt.savefig(_fname, bbox_inches='tight', dpi=DPI)
with open(str(_fname) + '.caption.md', 'w') as f:
    f.write(_caption)
    print(_caption)

### Normality

The analyses below will assume that the log2-transformed data is approximately normal. 

We can qualititatively verify this by plotting the data distributions.

We see that the fit is not perfect but is perhaps acceptable. 
We will therefore not worry about the normality of the data.

In [ ]:
from scipy.stats import norm
def _plot_norm_fit(values, *args, color=None, **kwargs):
    """
    Fits a normal distribution to the data and plots its PDF
    """
    mu, std = norm.fit(values)

    min_ = values.min()
    max_ = values.max()
    
    ax = plt.gca()
    xs = np.linspace(min_, max_, 100)
    ys = norm.pdf(xs, mu, std)
    
    ax.plot(xs, ys, **kwargs, color='black', label='Normal fit')
    ax.axvline(mu, color='black')
    
    ax.text(
        0.98, 0.98, '\n'.join([r'$\mu = {:.2f}$'.format(mu),r'$\sigma = {:.2f}$'.format(std)]),
        ha='right', va='top', transform=ax.transAxes
    )
    
    

# Convert data to long format
_df = data_numeric_log2.copy()
_df = _df.stack(_df.columns.names)
_df.name = 'value'
_df = _df.reset_index()

# Plot the histograms and normal distribution fits
fgrid = sns.FacetGrid(col='Experiment_Replicate', col_wrap=3, col_order=data_numeric_log2.columns, data=_df, size=FIVE_MM_IN_INCH*5)
fgrid.map(sns.histplot, 'value', stat='density')
fgrid.map(_plot_norm_fit, 'value')
fgrid.set_titles('{col_name}')

plt.suptitle("Normal fit of log2-transformed, unnormalised, and unfiltered data", y=1.05)

_fname = OUTPUT_DIRECTORY / '01-EDA-log2-transformed-unnormalised-unfiltered-normal-fit.pdf'
_caption = """
Distribution of non-null values of log2-transformed, unnormalised, and unfiltered dataset. 
The rows facet the four experiment types, while the columns split the data per replicate.
The green distribution plot shows the histogram of observed log2 values in the data.
The black curve shows the corresponding normal distribution fit to the data, whose parameters are written in the top-right corner.
The black vertical line indicates the mean ($\mu$) estimate.
"""
plt.savefig(_fname, bbox_inches='tight', dpi=DPI)
with open(str(_fname) + '.caption.md', 'w') as f:
    f.write(_caption)
    print(_caption)

### PCA 

And finally the PCA of unnormalissed data

In [ ]:
from adjustText import adjust_text
from sklearn.decomposition import PCA

pca = PCA(n_components=2)

# Note that we only look at the rows that are all not null:
projected = pca.fit_transform(data_numeric_log2.dropna(axis=0).T)

projected = pd.DataFrame(projected, index=data_numeric_log2.columns, columns=['PC1', 'PC2'])
explained_variance = pd.Series(pca.explained_variance_ratio_, index=projected.columns)

ax = plt.gca()

sns.scatterplot(x='PC1', y='PC2', hue='Experiment', data=projected.join(headers))

texts = []
for ix, row in projected.iterrows():
    texts.append(ax.text(row['PC1'], row['PC2'], ix, ha='center', va='center'))

xlim = ax.get_xlim()
ylim = ax.get_ylim()

# Increasing ylim helps place the H3K4me3 labels
ax.set_ylim(ylim[0]*1.1, ylim[1] * 1.1)

adjust_text(texts, projected['PC1'].values, projected['PC2'].values, arrowprops=dict(arrowstyle='-'))

ax.set_xlabel("PC1 ({:.2%} variance)".format(explained_variance.loc['PC1']))
ax.set_ylabel("PC2 ({:.2%} variance)".format(explained_variance.loc['PC2']))
ax.grid(False)
ax.set_title("PCA, log2-transformed data, unfiltered,\nunnormalised, only proteins with no missing values")
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), title='Experiment')


_fname = OUTPUT_DIRECTORY / '01-EDA-log2-transformed-unnormalised-unfiltered-PCA.pdf'
_caption = """
Principal Component embedding of log2-transformed, unnormalised, and unfiltered dataset, including only the rows which have no missing values.
The principal component 1 (PC1) is plotted on the X axis, while the PC2 is plotted on the Y. 
The explained variance ratio is highlighted in parentheses.

Note that the experiment types (colour, see legend on the right) do not cluster together as this data is unnormalised.
"""
plt.savefig(_fname, bbox_inches='tight', dpi=DPI)
with open(str(_fname) + '.caption.md', 'w') as f:
    f.write(_caption)
    print(_caption)

## Normalisation

For normalisation of these intensities, Andrey suggests the use of Histones H4 and H2B, namely:

- `Histone H4 OS=Homo sapiens OX=9606 GN=H4C1 PE=1 SV=2`
- `Histone H2B type 1-C/E/F/G/I OS=Homo sapiens OX=9606 GN=H2BC4 PE=1 SV=4`

In our transformed data these correspond to indices `H4C1` and `H2BC4` (from the `GN=` annotation in the description)

In subsequent analyses I have found that using more than these two proteins performs a bit better.
Here's the list of all proteins with the wort `histone` in description, that we observe in our dataset:

In [ ]:
data_metadata[data_metadata['Description'].str.lower().str.contains('histone')].sort_index()

Out of which the list `HISTONES` contains the set of core histone proteins and their variants

In [ ]:
HISTONES = [
    'H2AC20', 
    'H2AC21', 
    'H2AW', 
    'H2AZ2', 
    'H2BC4',
    'H2BU1', 
    'H3-2', 
    'H4C1',
    'MACROH2A1', 
    'MACROH2A2'
]

# # Andrey suggests using these:
# NORMALISATION_PROTEINS = ['H4C1', 'H2BC4']

# I think perhaps it might be better to use the full collection of histone
# proteins instead:

NORMALISATION_PROTEINS = HISTONES

In [ ]:
data_numeric_log2_histones = data_numeric_log2.loc[HISTONES]

In [ ]:
data_numeric_log2_histones

In [ ]:
_cmap = sns.clustermap(
    data_numeric_log2_histones,
    cmap='viridis', robust=True, 
    figsize=(20*FIVE_MM_IN_INCH, 10*FIVE_MM_IN_INCH),
    metric='correlation',
    method='complete',
)
_cmap.ax_col_dendrogram.set_title("Unnormalised data (log2) for histone proteins only")

for tick in _cmap.ax_heatmap.get_yticklabels():
    tick_text = tick.get_text()
    if tick_text in NORMALISATION_PROTEINS:
        tick.set_color('red')
        
_fname = OUTPUT_DIRECTORY / '02-normalisation-histones-heatmap-unnormalised.pdf'
_caption = """
Boxplot showing the unnormalised log2 data for histone proteins only.
The proteins are plotted on the Y axis, and the experiments on the X.

Proteins selected as normalisation factors are highlighted in red.
"""
plt.savefig(_fname, bbox_inches='tight', dpi=DPI)
with open(str(_fname) + '.caption.md', 'w') as f:
    f.write(_caption)
    print(_caption)

Mark `NORMALISATION_PROTEINS` as "Used for normalisation"

In [ ]:
comments_norm = pd.Series(None, dtype=object, index=data_numeric_log2.index)
comments_norm.loc[NORMALISATION_PROTEINS] = 'Used for normalisation'

Andrey used the sum of the (natural scale) intensities of these proteins as a normalisation factor;
we tried working on a similar note and use the mean of the log2-transformed intensities for such histone proteins,
however taking a median of M-offsets below seems to be a more robust estimator, the M-offsets can obtained by subtracting the row-wise mean from the data:

In [ ]:
data_numeric_log2.sub(data_numeric_log2.mean(axis=1), axis=0)

### Normalisation factors calculation

We will now compute the MA statistics from the unnormalised data.

MA statistics are simply row-wise averages of the data (`avg` column, the `A` part)
And the log2 offsets from this average (`log2_diff` column, the `M` part)

In [ ]:
data_numeric_log2_row_averages = data_numeric_log2.mean(axis=1)
data_numeric_log2_row_averages.name = 'avg'
data_numeric_log2_row_averages

In [ ]:
data_numeric_log2

In [ ]:
data_numeric_log2_diffs = data_numeric_log2.sub(data_numeric_log2_row_averages, axis=0)
data_numeric_log2_diffs

In [ ]:
data_numeric_log2_ma = data_numeric_log2_diffs.stack()
data_numeric_log2_ma.name = 'log2_diff'
data_numeric_log2_ma = data_numeric_log2_ma.reset_index().join(data_numeric_log2_row_averages, on='Label')
data_numeric_log2_ma

We will now compute the normalisation factors from the `NORMALISATION_PROTEINS`.

Andrey uses the sum of natural scale intensities, 
I have tried using the median of intensities (see below)

But I now think that it is more appropriate to take the median of the M offsets of the `NORMALISATION_PROTEINS`

In [ ]:
# Andrey's normfactors
# normalisation_factors = (data_numeric_log2.loc[NORMALISATION_PROTEINS].apply(lambda x: np.power(2,x)).sum()).apply(np.log2)

# My normfactors (attempt):
# normalisation_factors = data_numeric_log2.loc[NORMALISATION_PROTEINS].median()

# Get the normfactors from MA offsets
_ma_data_for_normalisation_proteins = data_numeric_log2_ma[
    data_numeric_log2_ma['Label'].isin(NORMALISATION_PROTEINS)
]
print(_ma_data_for_normalisation_proteins.shape)
assert set(_ma_data_for_normalisation_proteins['Label'].unique()) == set(NORMALISATION_PROTEINS)


normalisation_factors = _ma_data_for_normalisation_proteins.groupby('Experiment_Replicate')['log2_diff'].median()

normalisation_factors

In [ ]:
_ma_data_for_normalisation_proteins

### Plots (Before)

#### MA

To illustrate the normalisation behaviour it is helpful to make some MA plots:

In [ ]:
from adjustText import adjust_text
def annotate_histones(x, y, *, color=None, data=None):
    data_histones = data[data['Label'].isin(HISTONES)]
    ax = plt.gca()
    
    print(data_histones.shape)
    ax.scatter(data_histones[x], data_histones[y], color=color)
    
    texts = []
    for __, row in data_histones.iterrows():
        texts.append(ax.text(row[x], row[y], row['Label'], color=color))
        
    adjust_text(texts, arrowprops=dict(arrowstyle='-'), ax=ax)
    
def add_horizontal_line(*args, color=None, data=None):
    ax = plt.gca()
    ax.axhline(0, linestyle='-',color='black', zorder=1)

fgrid = sns.FacetGrid(col='Experiment_Replicate', col_wrap=3,
                      size=FIVE_MM_IN_INCH*10,
                      col_order=data_numeric_log2.columns, data=data_numeric_log2_ma)
fgrid.map(sns.scatterplot, 'avg', 'log2_diff', alpha=.4, color='black')
fgrid.map_dataframe(annotate_histones, 'avg', 'log2_diff')
fgrid.map_dataframe(add_horizontal_line)
fgrid.set_titles('{col_name}')

for experiment, ax in fgrid.axes_dict.items():
    ax.axhline(normalisation_factors.loc[experiment], color='r', linestyle='-')
    ax.grid(False)

plt.suptitle("MA plots of of log2-transformed, unnormalised, and unfiltered data", y=1.05)

_fname = OUTPUT_DIRECTORY / '02-normalisation-log2-transformed-unnormalised-unfiltered-ma-plot.pdf'
_caption = """
MA-like plot for log2-transformed, unnormalised and unfiltered data.

X axis plots mean(log2_intensity) across data rows (proteins). X axis is the same in every plot.
Y axis plots log2_intensity - mean(log2_intensity) offsets for individual samples

Histone proteins are highlighted in green.
Red line indicates the normalisation factor estimate.
"""
plt.savefig(_fname, bbox_inches='tight', dpi=DPI)
with open(str(_fname) + '.caption.md', 'w') as f:
    f.write(_caption)
    print(_caption)

#### Boxplot

Alternatively, we can plot all of this in a boxplot format:

In [ ]:
_df = data_numeric_log2.stack(list(data_numeric_log2.columns.names)).copy()
_df.name = 'log2_intensity_unnormalised'
_df = _df.reset_index()

_df['is_histone'] = False
_df.loc[_df['Label'].isin(HISTONES), 'is_histone'] = True

_df_cov = pd.DataFrame(
    # The + below recentres the normalisation factors around the mean of normalisation proteins
    {'normalisation_factors': normalisation_factors + data_numeric_log2.loc[NORMALISATION_PROTEINS].mean(axis=1).mean()}, 
)

fig = plt.figure(figsize=(10*FIVE_MM_IN_INCH, 15*FIVE_MM_IN_INCH))
ax = plt.gca()

sns.pointplot(
    y='Experiment_Replicate', x='normalisation_factors', 
    data=_df_cov.reset_index(),
    order=data_numeric_log2.columns,
    marker='.',
    color='black',
    label="Estimated covariates",
)

sns.boxplot(
    y='Experiment_Replicate', x='log2_intensity_unnormalised', 
    data=_df, hue='is_histone', showfliers=False,
    order=data_numeric_log2.columns,
)


ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), title="Histone protein")
ax.set_xlabel("Unnormalised log2 intensity")
ax.set_ylabel("Experiment, Replicate")
ax.set_title("Distribution of intensities, log2-transformed,\nunnormalised, unfiltered data")

_fname = OUTPUT_DIRECTORY / '02-normalisation-boxplot.pdf'
_caption = """
Boxplot showing the distribution of signal intensities for non-histone proteins (green), and the histone proteins (orange), prior to normalisation.
The intensity is plotted on the X axis, while the samples are facetted on the Y. The outliers ("fliers") are hidden.

Black line shows the estimated normalisation offsets centred around the mean of normalisation proteins.
"""
plt.savefig(_fname, bbox_inches='tight', dpi=DPI)
with open(str(_fname) + '.caption.md', 'w') as f:
    f.write(_caption)
    print(_caption)

### The bit where normalisation is performed

To perform the normalisation we simply subtract the normalisation factors from the dataset.

Note that as we are using (zero-centred) offsets for the normalisation, the intuitive meaning of the normalised intensities stays more or less the same.

In [ ]:
data_numeric_log2_normalised = data_numeric_log2 - normalisation_factors
# In the row below we need to make sure column order stays the same
data_numeric_log2_normalised = data_numeric_log2_normalised[data_numeric_log2.columns]

In [ ]:
from numpy.testing import assert_array_equal

# The minus operator above works columnwise, we can confirm this
for col in data_numeric_log2_normalised:
    assert_array_equal(data_numeric_log2_normalised[col], data_numeric_log2[col] - normalisation_factors.loc[col])

The normalisation has the effect of making the normalisation protein M-offsets constant
(though not necessarily zero, as median is used, not mean):

In [ ]:
data_numeric_log2_normalised.sub(data_numeric_log2_normalised.mean(axis=1), axis=0).loc[NORMALISATION_PROTEINS]

In [ ]:
data_numeric_log2_normalised.sub(data_numeric_log2_normalised.mean(axis=1), axis=0).loc[NORMALISATION_PROTEINS].median()

### Plots (After)

Let's see whether the normalisation improved things

#### Boxplot

In [ ]:
_df = data_numeric_log2_normalised.stack(list(data_numeric_log2.columns.names)).copy()
_df.name = 'log2_intensity_normalised'
_df = _df.reset_index()

_df['is_histone'] = False
_df.loc[_df['Label'].isin(HISTONES), 'is_histone'] = True


fig = plt.figure(figsize=(10*FIVE_MM_IN_INCH, 15*FIVE_MM_IN_INCH))
ax = plt.gca()

sns.boxplot(
    y='Experiment_Replicate', x='log2_intensity_normalised', 
    data=_df, hue='is_histone', showfliers=False,
    order=data_numeric_log2.columns,
)


ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), title="Histone protein")
ax.set_xlabel("Normalised log2 intensity")
ax.set_ylabel("Experiment, Replicate")
ax.set_title("Distribution of intensities, log2-transformed,\nnormalised, unfiltered data")

_fname = OUTPUT_DIRECTORY / '02-normalisation-boxplot-post-normalisation.pdf'
_caption = """
Boxplot showing the distribution of signal intensities for non-histone proteins (green), and the histone proteins (orange), after  normalisation by the mean of histone protein intensities.
The intensity is plotted on the X axis, while the samples are facetted on the Y. The outliers ("fliers") are hidden.
"""
plt.savefig(_fname, bbox_inches='tight', dpi=DPI)
with open(str(_fname) + '.caption.md', 'w') as f:
    f.write(_caption)
    print(_caption)

#### Histone heatmap

Let's replot histones after normalisation

In [ ]:
_cmap = sns.clustermap(
    data_numeric_log2_normalised.loc[HISTONES],
    cmap='viridis', robust=True, 
    figsize=(20*FIVE_MM_IN_INCH, 10*FIVE_MM_IN_INCH),
    metric='correlation',
    method='complete',
)
_cmap.ax_col_dendrogram.set_title("Normalised data (log2) for histone proteins only")

for tick in _cmap.ax_heatmap.get_yticklabels():
    tick_text = tick.get_text()
    if tick_text in NORMALISATION_PROTEINS:
        tick.set_color('red')
        
_fname = OUTPUT_DIRECTORY / '02-normalisation-histones-heatmap-post-normalisation.pdf'
_caption = """
Boxplot showing the unnormalised log2 data for histone proteins only.
The proteins are plotted on the Y axis, and the experiments on the X.

Proteins selected as normalisation factors are highlighted in red.
"""
plt.savefig(_fname, bbox_inches='tight', dpi=DPI)
with open(str(_fname) + '.caption.md', 'w') as f:
    f.write(_caption)
    print(_caption)

#### MA

And the MA plot

In [ ]:
data_numeric_log2_normalised_row_averages = data_numeric_log2_normalised.mean(axis=1)
data_numeric_log2_normalised_row_averages.name = 'avg'
data_numeric_log2_normalised_row_averages

In [ ]:
data_numeric_log2_normalised_diffs = data_numeric_log2_normalised.sub(data_numeric_log2_normalised_row_averages, axis=0)

In [ ]:
data_numeric_log2_normalised_ma = data_numeric_log2_normalised_diffs.stack()
data_numeric_log2_normalised_ma.name = 'log2_diff'
data_numeric_log2_normalised_ma = data_numeric_log2_normalised_ma.reset_index().join(data_numeric_log2_normalised_row_averages, on='Label')
data_numeric_log2_normalised_ma

In [ ]:
fgrid = sns.FacetGrid(col='Experiment_Replicate', col_wrap=3,
                      size=FIVE_MM_IN_INCH*10,
                      col_order=data_numeric_log2.columns, data=data_numeric_log2_normalised_ma)
fgrid.map(sns.scatterplot, 'avg', 'log2_diff', alpha=.4, color='black')
fgrid.map_dataframe(annotate_histones, 'avg', 'log2_diff')
# fgrid.map_dataframe(add_horizontal_line)
fgrid.set_titles('{col_name}')
for experiment, ax in fgrid.axes_dict.items():
    ax.axhline(0, linestyle='-', color='red', zorder=1)
    ax.grid(False)

plt.suptitle("MA plots of of log2-transformed, normalised, and unfiltered data", y=1.05)

_fname = OUTPUT_DIRECTORY / '02-normalisation-log2-transformed-normalised-unfiltered-ma-plot.pdf'
_caption = """
MA-like plot for log2-transformed, normalised and unfiltered data.

X axis plots mean(log2_intensity) across data rows (proteins). X axis is the same in every plot.
Y axis plots log2_intensity - mean(log2_intensity) offsets for individual samples

Histone proteins are highlighted in green.

Red line indicates x=0 line around which the data is centered now.
"""
plt.savefig(_fname, bbox_inches='tight', dpi=DPI)
with open(str(_fname) + '.caption.md', 'w') as f:
    f.write(_caption)
    print(_caption)

#### Correlation matrix

In [ ]:
sns.clustermap(data_numeric_log2_normalised.corr(), 
               cmap='viridis', 
               annot=True, fmt='.2f',
               figsize=(FIVE_MM_IN_INCH*20, FIVE_MM_IN_INCH*20))

_fname = OUTPUT_DIRECTORY / '02-correlation-matrix-of-normalised-data.pdf'
_caption = """
A Pearson correlation matrix showing the coerrelation coefficient estimate,
for the datasets, after normalisation. 
"""
plt.savefig(_fname, bbox_inches='tight', dpi=DPI)
with open(str(_fname) + '.caption.md', 'w') as f:
    f.write(_caption)
    print(_caption)

#### PCA

We can also observe how the PCA plot changes after the normalisation

In [ ]:
from adjustText import adjust_text
from sklearn.decomposition import PCA

pca = PCA(n_components=2)

# Note that we only look at the rows that are all not null:
projected = pca.fit_transform(data_numeric_log2_normalised.dropna(axis=0).T)

projected = pd.DataFrame(projected, index=data_numeric_log2_normalised.columns, columns=['PC1', 'PC2'])
explained_variance = pd.Series(pca.explained_variance_ratio_, index=projected.columns)

ax = plt.gca()

sns.scatterplot(x='PC1', y='PC2', hue='Experiment', data=projected.join(headers))

texts = []
for ix, row in projected.iterrows():
    texts.append(ax.text(row['PC1'], row['PC2'], ix, ha='center', va='center'))

xlim = ax.get_xlim()
ylim = ax.get_ylim()

# Increasing ylim helps place the H3K4me3 labels
ax.set_ylim(ylim[0]*1.1, ylim[1] * 1.1)

adjust_text(texts, projected['PC1'].values, projected['PC2'].values, arrowprops=dict(arrowstyle='-'))

ax.set_xlabel("PC1 ({:.2%} variance)".format(explained_variance.loc['PC1']))
ax.set_ylabel("PC2 ({:.2%} variance)".format(explained_variance.loc['PC2']))
ax.grid(False)
ax.set_title("PCA, log2-transformed data, unfiltered,\nnormalised, only proteins with no missing values")
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), title='Experiment')


_fname = OUTPUT_DIRECTORY / '02-normalisation-PCA-post-normalisation.pdf'
_caption = """
Principal Component embedding of log2-transformed, normalised, and unfiltered dataset, including only the rows which have no missing values.
The principal component 1 (PC1) is plotted on the X axis, while the PC2 is plotted on the Y. 
The explained variance ratio is highlighted in parentheses.

Note that the experiment types (colour, see legend on the right) cluster together much better after the normalisation.
"""
plt.savefig(_fname, bbox_inches='tight', dpi=DPI)
with open(str(_fname) + '.caption.md', 'w') as f:
    f.write(_caption)
    print(_caption)

## Filtering

It is counterproductive to throw _all_ of the data into the model.
For better performance we will filter out some of the proteins.

Particularly, we will only keep the proteins which have at least `MINIMUM_NUMBER_OF_OBSERVED_VALUES_PER_CONDITION` in any of the four experimental conditions (H3, H4, H3K4me1, or H3K4me3).

In [ ]:
MINIMUM_NUMBER_OF_OBSERVED_VALUES_PER_CONDITION

First count how many non-null values does each protein have:

In [ ]:
# Make a long-format DF with non-null values
_df = (~data_numeric_log2_normalised.isnull()).stack()
_df.name = 'is_not_null'

# Join with headers
_df = _df.reset_index().join(headers, on='Experiment_Replicate')

# Groupby label and experiment and count
number_non_null_per_condition = _df.groupby(['Label', 'Experiment'])['is_not_null'].sum().unstack('Experiment')
number_non_null_per_condition

Now check which labels match our condition:

In [ ]:
sufficient_number_of_values = (number_non_null_per_condition >= MINIMUM_NUMBER_OF_OBSERVED_VALUES_PER_CONDITION).any(axis=1)
sufficient_number_of_values

We will throw away all of the proteins which have the value set to False, in hte series above. There are this many of them:

In [ ]:
sufficient_number_of_values.value_counts()

In order to stay civilised we will create a new series called "Comment" and mark the reason of filtering out in there:

In [ ]:
data_comment = pd.Series(None, index=data_numeric_log2_normalised.index, name='Comment', dtype=str)
data_comment.loc[~sufficient_number_of_values] = "Insufficient number of non-null values"

In [ ]:
data_comment

In [ ]:
data_comment.value_counts()

Perfect, we can move on to modelling

## Modeling

### The inputs

We will model the only the normalised data which has a sufficient number of values:

In [ ]:
data_to_model = data_numeric_log2_normalised.loc[sufficient_number_of_values]
data_to_model

This is how the missing value distribution for this data looks like

In [ ]:
import missingno as msgno
msgno.matrix(data_to_model, figsize=(20*FIVE_MM_IN_INCH, 20*FIVE_MM_IN_INCH), fontsize=6)

plt.title("Distribution of missing values in log2-transformed, normalised, and filtered data")
plt.xlabel("Data columns (experiments)")
plt.ylabel("Data rows (proteins)")
plt.xticks(rotation=90)

_fname = OUTPUT_DIRECTORY / '03-modelling-log2-transformed-normalised-filtered-missing-values.png'
_caption = """
Distribution of missing values in the log2-transformed, normalised, and filtered data which will be used for modelling. 
The heatmap above displays all  929 proteins remaining in the data after filtering in rows, and all experiments in columns. 
Dark values indicate the presence of a value, while white space indicates the absence of one. 
The sparkline on the right hand side counts the number of values per protein, which ranges from zero to 12 in this dataset.
"""
plt.savefig(_fname, bbox_inches='tight', dpi=DPI)
with open(str(_fname) + '.caption.md', 'w') as f:
    f.write(_caption)
    print(_caption)

The special cases here are the proteins which have all zero values in some of the conditions:

In [ ]:
all_zero_in_some_condition = (number_non_null_per_condition == 0).any(axis=1)
all_zero_in_some_condition = all_zero_in_some_condition.loc[data_to_model.index]
all_zero_in_some_condition.value_counts()

These proteins are special cases. To visualise them better let's establish a consistent sort order
which is based on which condition is missing

In [ ]:
all_zero_in_some_condition_sort_order = number_non_null_per_condition.loc[all_zero_in_some_condition[all_zero_in_some_condition].index].sort_values(by=['H3', 'H4', 'H3K4me1', 'H3K4me3']).index

Let's plot these special cases

In [ ]:
import missingno as msgno
msgno.matrix(
    data_to_model.loc[all_zero_in_some_condition_sort_order], figsize=(20*FIVE_MM_IN_INCH, 20*FIVE_MM_IN_INCH), fontsize=6)

plt.title("Distribution of missing values in log2-transformed, normalised, and filtered data\nshown only the special cases with all-nulls in some of the conditions")
plt.xlabel("Data columns (experiments)")
plt.ylabel("Data rows (proteins)")
plt.xticks(rotation=90)

# For some reason this plot does not rasterize well..
_fname = OUTPUT_DIRECTORY / '03-modelling-log2-transformed-normalised-filtered-missing-values-all_zero_in_some_condition.png'
_caption = """
Distribution of missing values in the log2-transformed, normalised, and filtered data of proteins that correspond to "special cases" in the modeling as they have all zero values in at least one of the conditions.
The heatmap above displays all 102 of such proteins, sorted by their missing value pattern. 
Dark values indicate the presence of a value, while white space indicates the absence of one. 
The sparkline on the right hand side counts the number of values per protein, which ranges from zero to 12 in this dataset.
"""
plt.savefig(_fname, bbox_inches='tight', dpi=DPI)
with open(str(_fname) + '.caption.md', 'w') as f:
    f.write(_caption)
    print(_caption)

We can also get a few special cases of proteins from that list

In [ ]:
number_non_null_per_condition

In [ ]:
number_non_null_per_condition.loc[
    (number_non_null_per_condition[['H3', 'H4']] > 2).any(axis=1) & (number_non_null_per_condition[['H3K4me1', 'H3K4me3']] == 0).all(axis=1)
]

In [ ]:
REPRESENTATIVE_NULLS = [
    'FAM98A', 'PHF8', 'EIF4G1', 'ZMYND11', # Controls null, proteins non-null
    'PSMC1', 'SNTB2', 'TUBB2A', 'TNNT3', # Controls non-null, proteins null
]

In [ ]:
# Convert data to long format
_df = data_to_model.copy()
_df = _df.stack(_df.columns.names)
_df.name = 'value'
_df = _df.reset_index()

# Plot the histograms and normal distribution fits
fgrid = sns.FacetGrid(col='Experiment_Replicate', col_wrap=3, col_order=data_to_model.columns, data=_df, size=FIVE_MM_IN_INCH*5)
fgrid.map(sns.histplot, 'value', stat='density')
fgrid.map(_plot_norm_fit, 'value')
fgrid.set_titles('{col_name}')

plt.suptitle("Normal fit of log2-transformed, normalised, and filtered data", y=1.05)

_fname = OUTPUT_DIRECTORY / '03-modelling-log2-transformed-normalised-filtered-normal-fit.pdf'
_caption = """
Distribution of non-null values of log2-transformed, normalised, and filtered dataset. 
The rows facet the four experiment types, while the columns split the data per replicate.
The green distribution plot shows the histogram of observed log2 values in the data.
The black curve shows the corresponding normal distribution fit to the data, whose parameters are written in the top-right corner.
The black vertical line indicates the mean ($\mu$) estimate.
"""
plt.savefig(_fname, bbox_inches='tight', dpi=DPI)
with open(str(_fname) + '.caption.md', 'w') as f:
    f.write(_caption)
    print(_caption)

### Design matrix and contrasts

For modeling we will use limma and encode the data as a "means" model.

We first need to load the limma into `rpy2` environment:

In [ ]:
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri, numpy2ri

%load_ext rpy2.ipython

Time to load data into R, we will use `limma` for most tastks:

In [ ]:
%%R

library("limma")
packageVersion("limma")

Now we load the header data and make sure that it's columns are interpreted as factors correctly:

In [ ]:
assert headers.index.equals(data_to_model.columns), "Make sure that headers index matches data_to_model columns!"

In [ ]:
%%R -i headers


headers$Experiment <- as.factor(headers$Experiment)
headers$Experiment <- relevel(headers$Experiment, ref="H4")

headers$Replicate <- as.factor(headers$Replicate)
headers

To create the design matrix, we will use a Means (no intercept) design 

In [ ]:
%%R

# Do not use intercept:
design <- model.matrix(~0 + Experiment, headers)
design

From this design matrix we are primarily interested in three contrasts:

- `H3K4me1vsControl`: `H3K4me1` vs mean of `H4` and `H3` controls
- `H3K4me3vsControl`: `H3K4me3` vs mean of `H4` and `H3` controls
- `H3K4me3vsH3K4me1`: `H3K4me3` vs `H3K4me1` (note that this is the same as `H3K4me1vsControl` vs `H3K4me3vsControl`)

In [ ]:
%%R -o contrasts_matrix_as_df

contrasts.matrix <- limma::makeContrasts(
    "H3K4me1vsControl"=ExperimentH3K4me1-(ExperimentH4 + ExperimentH3)/2,
    "H3K4me3vsControl"=ExperimentH3K4me3-(ExperimentH4 + ExperimentH3)/2, 
    "H3K4me3vsH3K4me1"=ExperimentH3K4me3-ExperimentH3K4me1, 
    
    levels=design
)

contrasts_matrix_as_df = as.data.frame(contrasts.matrix)
contrasts.matrix

In [ ]:
contrasts_matrix_as_df

### The fit

Here we fit the simple limma model, and extract the resulting coefficient estimates and contrasts.
Note that we use `robust=TRUE` in the `eBayes` function

In [ ]:
%%R -i data_to_model -o coef_estimates_design -o coef_estimates_contrasts


fit <- limma::lmFit(data_to_model, design)

coef_estimates_design = as.data.frame(fit$coefficients)

fit2 <- limma::contrasts.fit(fit, contrasts.matrix)

# Note robust=TRUE
fit2 <- limma::eBayes(fit2, robust=TRUE)

coef_estimates_contrasts <- as.data.frame(fit2$coefficients)


For coefficient estimates it is now most important to check that the coefficients were estimated in a predictable way for those special case proteins that have all nulls in at least one of the conditions.

In [ ]:
_df = number_non_null_per_condition.loc[data_to_model.index]

fraction_null_coefs = []
# For each experiment
for experiment in _df.columns:
    # For each number of non-null values possible
    for non_null_count, _subdf in _df[experiment].groupby(_df[experiment]):
        # Count number of null coefficient_estimates for the coefficient
        fraction_null_coefs.append([experiment, non_null_count, coef_estimates_design.loc[_subdf.index, f'Experiment{experiment}'].isnull().astype(float).mean()])
        
fraction_null_coefs = pd.DataFrame(fraction_null_coefs, columns=['Experiment',' Number of non-null observations', 'Fraction of null coefficient estimates'])
fraction_null_coefs
        

Note that the coefficients are always null when there is zero data for this particular condition (fraction=1)
But always non-null otherwise

Below is avisualisation of a few representative proteins:

In [ ]:
data_to_model.loc[REPRESENTATIVE_NULLS]

In [ ]:
coef_estimates_design.loc[REPRESENTATIVE_NULLS, ['ExperimentH3', 'ExperimentH4', 'ExperimentH3K4me3', 'ExperimentH3K4me1']]

And the corresponding contrasts:

In [ ]:
coef_estimates_contrasts.loc[REPRESENTATIVE_NULLS]

## Results

The results can be extracted from R by calling the `topTable` function in limma once for each contrast, and setting `number` parameter to the number of rows in the data:

In [ ]:
%%R -o results_r

coefs <- colnames(contrasts.matrix)
results_r <- list()

for (i in 1:length(coefs)) {
    coef <- coefs[[i]]
    results_r[[coef]] <- limma::topTable(fit2, coef=coef, adjust.method="BH", number=nrow(data_to_model))
}


We convert these results to python and add another column called "Significant" which is true if the adjusted p value is lower than `FDR_THRESHOLD` specified in parameters

In [ ]:
FDR_THRESHOLD

In [ ]:
with robjects.conversion.localconverter(robjects.default_converter + pandas2ri.converter) as co:
    results = {
        k: co.rpy2py(v) for k,v in results_r.items()
    }

for k in results.keys():
    results[k]['significant'] = results[k]['adj.P.Val'] <= FDR_THRESHOLD
    
results = pd.concat(results, keys=results.keys(), axis=1)
results

In [ ]:
assert set(results.index) == set(data_to_model.index), "Results index does not match input index"

In [ ]:
results.loc(axis=1)[:, 'significant'].sum()

### Sanity checks

#### Intuitive interpretation of contrasts

We now check these results to see that the estimated log fold changes (by limma) are close to what we would expect from naive modelling. 

This is needed both as a sanity check, and as a justification for "fold change imputation" for a few cases where the FCs could not be estimated analytically.

In [ ]:
data_to_model_by_experiment = {
    'H3': data_to_model[['H3_1', 'H3_2', 'H3_3']],
    'H4': data_to_model[['H4_1', 'H4_2', 'H4_3']],
    'H3K4me1': data_to_model[['H3K4me1_1', 'H3K4me1_2', 'H3K4me1_3']],
    'H3K4me3': data_to_model[['H3K4me3_1', 'H3K4me3_2', 'H3K4me3_3']],
    'Controls': data_to_model[['H3_1', 'H3_2', 'H3_3', 'H4_1', 'H4_2', 'H4_3']],
}

In [ ]:
_controls = 0.5 * (data_to_model_by_experiment['H3'].mean(axis=1) +  data_to_model_by_experiment['H4'].mean(axis=1))
expected_naively = {
    'H3K4me1vsControl': data_to_model_by_experiment['H3K4me1'].mean(axis=1) - _controls,
    'H3K4me3vsControl': data_to_model_by_experiment['H3K4me3'].mean(axis=1) - _controls,
    'H3K4me3vsH3K4me1': data_to_model_by_experiment['H3K4me3'].mean(axis=1) - data_to_model_by_experiment['H3K4me1'].mean(axis=1),
}

In [ ]:
_controls

In [ ]:
expected_naively['H3K4me3vsH3K4me1'].loc['PHF8']

In [ ]:
results['H3K4me3vsH3K4me1', 'logFC'].loc['PHF8']

import pingouin as pg

In [ ]:
import pingouin as pg

In [ ]:
for column, expectation in expected_naively.items():
    fig = plt.figure()
    ax = plt.gca()
    
    _df = pd.DataFrame({'expectation': expectation, 'actual': results[column, 'logFC']})
    
    correlation = pg.corr(_df['expectation'], _df['actual'], method='pearson')
    assert correlation.loc['pearson', 'r'] == 1 # This should always be one
    correlation = correlation.loc['pearson', 'CI95%']
    
    sns.regplot(x='expectation', y='actual', scatter_kws=dict(alpha=0.2), data=_df)
    
    ax.text(0.02, 0.98, r'$r \in [{}, {}]$'.format(*correlation), transform=ax.transAxes, ha='left', va='top')
    ax.set_xlabel("Expected result (calculated naively)")
    ax.set_ylabel("Actual result (limma)")
    ax.set_title(column)
    sns.despine(ax=ax, offset=5)
    
    ax.grid(False)
    
    _fname = OUTPUT_DIRECTORY / f'04-modelling-sanity-checks-for-coefficient-interpretation-{column}.pdf'
    _caption = f"""
    Comparison of the {column} coefficient estimates computed by the model (y axis), and their expected values (x axis).
    The expected values were computed naively - by subtracting the coresponding means from each other.
    The actual values were computed using the limma statistical model.
    
    The scatterpoint highlight individual protein estimates, the line is a linear regression estimate.
    The 95% confidence interval for pearson R estimate is written in the top left corner.
    
    If the model is working correctly, we would expect a perfect fit here.
    """
    plt.savefig(_fname, bbox_inches='tight', dpi=DPI)
    with open(str(_fname) + '.caption.md', 'w') as f:
        f.write(_caption)
        print(_caption)
    

It's also worth seeing how much these results would change if we took a naive mean of the controls, i.e. mean(H3_1,H3_2,H3_3,H4_1,H4_2,H4_3) as opposed to the mean-of-means approach, i.e. 0.5 (mean(H3_1,H3_2,H3_3) + mean(H4_1,H4_2,H4_3))

In [ ]:
_controls_simple = data_to_model_by_experiment['Controls'].mean(axis=1)
expected_naively_simple_mean_of_controls = {
    'H3K4me1vsControl': data_to_model_by_experiment['H3K4me1'].mean(axis=1) - _controls_simple,
    'H3K4me3vsControl': data_to_model_by_experiment['H3K4me3'].mean(axis=1) - _controls_simple,
}

In [ ]:
for column, expectation in expected_naively_simple_mean_of_controls.items():
    fig = plt.figure()
    ax = plt.gca()
    
    _df = pd.DataFrame({'expectation': expectation, 'actual': results[column, 'logFC']})
    
    correlation = pg.corr(_df['expectation'], _df['actual'], method='pearson').loc['pearson', 'CI95%']
    
    sns.regplot(x='expectation', y='actual', scatter_kws=dict(alpha=0.2), data=_df)
    
    ax.text(0.02, 0.98, r'$r \in [{}, {}]$'.format(*correlation), transform=ax.transAxes, ha='left', va='top')
    ax.set_xlabel("Expected result (calculated naively, using simple mean of controls)")
    ax.set_ylabel("Actual result (limma)")
    ax.set_title(column)
    sns.despine(ax=ax, offset=5)
    
    ax.grid(False)
    
    _fname = OUTPUT_DIRECTORY / f'04-modelling-sanity-checks-for-coefficient-interpretation-{column}-simple-mean-of-controls.pdf'
    _caption = f"""
    Comparison of the {column} coefficient estimates computed by the model (y axis), and their simplified expected values (x axis).
    The expected values were computed naively - by subtracting the coresponding means from each other, and by taking a simple mean of the controls, as opposed to the mean-of-means used in the model.
    The actual values were computed using the limma statistical model.
    
    The scatterpoint highlight individual protein estimates, the line is a linear regression estimate.
    The 95% confidence interval for pearson R estimate is written in the top left corner.
    
    If the model is working correctly, we would expect a perfect fit here.
    """
    plt.savefig(_fname, bbox_inches='tight', dpi=DPI)
    with open(str(_fname) + '.caption.md', 'w') as f:
        f.write(_caption)
        print(_caption)
    

#### Heatmaps

##### H3K4me1 vs Control

In [ ]:
_colname = 'H3K4me1vsControl'
_significant = results.loc[results[_colname, 'significant'], _colname].sort_values(by='logFC', ascending=False)
_significant

In [ ]:
from matplotlib.gridspec import GridSpec
figure = plt.figure(figsize=(20*FIVE_MM_IN_INCH, 28*FIVE_MM_IN_INCH), constrained_layout=True)

gs = GridSpec(2, 2, figure=figure, width_ratios=(1, 12), height_ratios=(10,1))
ax_legends = figure.add_subplot(gs[1,:])
ax_legends.axis('off')
cax_left = ax_legends.inset_axes([0.02, 0.58, 0.2, 0.2], transform=ax_legends.transAxes)
cax_right = ax_legends.inset_axes([0.32, 0.58, 0.2, 0.2], transform=ax_legends.transAxes)

cax_left.set_title("LogFC estimate")
cax_right.set_title("Normalised data,\nH3/H4 centred")

ax_left = figure.add_subplot(gs[0, 0])
ax_right = figure.add_subplot(gs[0, 1], sharey=ax_left)


vmax = np.ceil(_significant['logFC'].abs().replace(np.inf, np.nan).dropna().quantile(0.97))
vmin = -vmax

_matrix = data_to_model.sub(data_to_model_by_experiment['Controls'].mean(axis=1), axis=0)
_matrix = _matrix.loc[_significant.index]

heatmap_right = sns.heatmap(
    _matrix, 
    cmap='RdBu_r', robust=True, center=0,
    ax=ax_right, yticklabels=0, 
    cbar=True,
    cbar_ax = cax_right,
    cbar_kws=dict(orientation='horizontal'),
    linewidth=0,
    linecolor='black',
)

heatmap_right.set_facecolor('#969696')

heatmap_left = sns.heatmap(
    _significant[['logFC']], 
    cmap='RdBu_r', 
    center=0, 
    robust=True,
    ax=ax_left, yticklabels=0, 
    cbar=True,
    cbar_ax=cax_left,
    cbar_kws=dict(orientation='horizontal'),
    linewidth=0,
    vmin=vmin, vmax=vmax,
    linecolor='black',
)

heatmap_right.xaxis.tick_top()
heatmap_right.xaxis.set_tick_params(length=0, rotation=90)

heatmap_left.xaxis.tick_top()
heatmap_left.xaxis.set_tick_params(length=0, rotation=90)

heatmap_right.yaxis.set_tick_params(length=0)
heatmap_left.yaxis.set_tick_params(length=0)

for ax in [heatmap_left, heatmap_right]:
    for spine in ax.spines:
        ax.spines[spine].set_visible(True)

for tick in heatmap_right.yaxis.get_ticklabels():
    tick.set_visible(False)

heatmap_right.set_title(f"Proteins with significant logFC estimates for {_colname}")

_fname = OUTPUT_DIRECTORY / f'05-modelling-heatmap-of-significant-proteins-for-{_colname}.pdf'
_caption = f"""

Heatmap of the significant `logFC` estimates for the {_colname} coefficient.
The proteins are sorted by the estimate, descending.

The left heatmap plots the `logFC` estimate, the colour axis limits are set to [{vmin}, {vmax}].
The right heatmap shows the data used for modelling, standardised to have zero mean of controls (H3 and H4). 

Missing values are marked in grey.
"""
plt.savefig(_fname, bbox_inches='tight', dpi=DPI)
with open(str(_fname) + '.caption.md', 'w') as f:
    f.write(_caption)
    print(_caption)


##### H3K4me3 vs Control

In [ ]:
_colname = 'H3K4me3vsControl'
_significant = results.loc[results[_colname, 'significant'], _colname].sort_values(by='logFC', ascending=False)
_significant

In [ ]:
from matplotlib.gridspec import GridSpec
figure = plt.figure(figsize=(20*FIVE_MM_IN_INCH, 28*FIVE_MM_IN_INCH), constrained_layout=True)

gs = GridSpec(2, 2, figure=figure, width_ratios=(1, 12), height_ratios=(10,1))
ax_legends = figure.add_subplot(gs[1,:])
ax_legends.axis('off')
cax_left = ax_legends.inset_axes([0.02, 0.58, 0.2, 0.2], transform=ax_legends.transAxes)
cax_right = ax_legends.inset_axes([0.32, 0.58, 0.2, 0.2], transform=ax_legends.transAxes)

cax_left.set_title("LogFC estimate")
cax_right.set_title("Normalised data,\nH3/H4 centred")

ax_left = figure.add_subplot(gs[0, 0])
ax_right = figure.add_subplot(gs[0, 1], sharey=ax_left)


vmax = np.ceil(_significant['logFC'].abs().replace(np.inf, np.nan).dropna().quantile(0.97))
vmin = -vmax

_matrix = data_to_model.sub(data_to_model_by_experiment['Controls'].mean(axis=1), axis=0)
_matrix = _matrix.loc[_significant.index]

heatmap_right = sns.heatmap(
    _matrix, 
    cmap='RdBu_r', robust=True, center=0,
    ax=ax_right, yticklabels=0, 
    cbar=True,
    cbar_ax = cax_right,
    cbar_kws=dict(orientation='horizontal'),
    linewidth=0,
    linecolor='black',
)
heatmap_right.set_facecolor('#969696')

heatmap_left = sns.heatmap(
    _significant[['logFC']], 
    cmap='RdBu_r', 
    center=0, 
    robust=True,
    ax=ax_left, yticklabels=0, 
    cbar=True,
    cbar_ax=cax_left,
    cbar_kws=dict(orientation='horizontal'),
    linewidth=0,
    vmin=vmin, vmax=vmax,
    linecolor='black',
)

heatmap_right.xaxis.tick_top()
heatmap_right.xaxis.set_tick_params(length=0, rotation=90)

heatmap_left.xaxis.tick_top()
heatmap_left.xaxis.set_tick_params(length=0, rotation=90)

heatmap_right.yaxis.set_tick_params(length=0)
heatmap_left.yaxis.set_tick_params(length=0)

for ax in [heatmap_left, heatmap_right]:
    for spine in ax.spines:
        ax.spines[spine].set_visible(True)

for tick in heatmap_right.yaxis.get_ticklabels():
    tick.set_visible(False)

heatmap_right.set_title(f"Proteins with significant logFC estimates for {_colname}")

_fname = OUTPUT_DIRECTORY / f'05-modelling-heatmap-of-significant-proteins-for-{_colname}.pdf'
_caption = f"""

Heatmap of the significant `logFC` estimates for the {_colname} coefficient.
The proteins are sorted by the estimate, descending.

The left heatmap plots the `logFC` estimate, the colour axis limits are set to [{vmin}, {vmax}].
The right heatmap shows the data used for modelling, standardised to have zero mean of controls (H3 and H4). 

Missing values are marked in grey.
"""
plt.savefig(_fname, bbox_inches='tight', dpi=DPI)
with open(str(_fname) + '.caption.md', 'w') as f:
    f.write(_caption)
    print(_caption)


##### H3K4me3 vs H3K4me1

In [ ]:
_colname = 'H3K4me3vsH3K4me1'
_significant = results.loc[results[_colname, 'significant'], _colname].sort_values(by='logFC', ascending=False)
_significant

In [ ]:
from matplotlib.gridspec import GridSpec
figure = plt.figure(figsize=(20*FIVE_MM_IN_INCH, 28*FIVE_MM_IN_INCH), constrained_layout=True)

gs = GridSpec(2, 2, figure=figure, width_ratios=(1, 12), height_ratios=(10,1))
ax_legends = figure.add_subplot(gs[1,:])
ax_legends.axis('off')
cax_left = ax_legends.inset_axes([0.02, 0.58, 0.2, 0.2], transform=ax_legends.transAxes)
cax_right = ax_legends.inset_axes([0.32, 0.58, 0.2, 0.2], transform=ax_legends.transAxes)

cax_left.set_title("LogFC estimate")
cax_right.set_title("Normalised data,\nminus mean(H3K4me1,H3K4me3)")

ax_left = figure.add_subplot(gs[0, 0])
ax_right = figure.add_subplot(gs[0, 1], sharey=ax_left)


vmax = np.ceil(_significant['logFC'].abs().replace(np.inf, np.nan).dropna().quantile(0.97))
vmin = -vmax

_matrix = data_to_model.sub(data_to_model_by_experiment['H3K4me1'].join(data_to_model_by_experiment['H3K4me3']).mean(axis=1), axis=0)
_matrix = _matrix.loc[_significant.index]

heatmap_right = sns.heatmap(
    _matrix, 
    cmap='RdBu_r', robust=True, center=0,
    ax=ax_right, yticklabels=1, 
    cbar=True,
    cbar_ax = cax_right,
    cbar_kws=dict(orientation='horizontal'),
    linewidth=0.1,
    linecolor='black',
)
heatmap_right.set_facecolor('#969696')
heatmap_left = sns.heatmap(
    _significant[['logFC']], 
    cmap='RdBu_r', 
    center=0, 
    robust=True,
    ax=ax_left, yticklabels=1, 
    cbar=True,
    cbar_ax=cax_left,
    cbar_kws=dict(orientation='horizontal'),
    linewidth=0.1,
    vmin=vmin, vmax=vmax,
    linecolor='black',
)

heatmap_right.xaxis.tick_top()
heatmap_right.xaxis.set_tick_params(length=0, rotation=90)

heatmap_left.xaxis.tick_top()
heatmap_left.xaxis.set_tick_params(length=0, rotation=90)

heatmap_right.yaxis.set_tick_params(length=0)
heatmap_left.yaxis.set_tick_params(length=0)

for ax in [heatmap_left, heatmap_right]:
    for spine in ax.spines:
        ax.spines[spine].set_visible(True)

for tick in heatmap_right.yaxis.get_ticklabels():
    tick.set_visible(False)

heatmap_right.set_title(f"Proteins with significant logFC estimates for {_colname}")

_fname = OUTPUT_DIRECTORY / f'05-modelling-heatmap-of-significant-proteins-for-{_colname}.pdf'
_caption = f"""

Heatmap of the significant `logFC` estimates for the {_colname} coefficient.
The proteins are sorted by the estimate, descending.

The left heatmap plots the `logFC` estimate, the colour axis limits are set to [{vmin}, {vmax}].
The right heatmap shows the data used for modelling, standardised to have mean(H3K4me3 and H3K4me1) = 0. 

Missing values are marked in grey.
"""
plt.savefig(_fname, bbox_inches='tight', dpi=DPI)
with open(str(_fname) + '.caption.md', 'w') as f:
    f.write(_caption)
    print(_caption)


## Imputation of results

While the model behaves correctly for most proteins, we want to:

1. flag the following result estimates:
   - estimates which are based only on one data point
   - imputed estimates (see below)
2. impute certain fold change estimates, namely:
   - mark proteins detected in treatments, but not detected in both controls as "infinitely enriched" (logFC=+inf)
   - mark proteins detected in both controls, but not in treatmets as "infinitely excluded" (logFC=-inf)
   - in cases where logFC could not be estimated because one (but not both!) controls are missing, calculate the logFC only from the control that is detected.

Make sure code below aligns with the contrasts matrix here:

In [ ]:
contrasts_matrix_as_df

In [ ]:
def flag_and_or_impute(estimate, treatments, controls):
    
    # Count number of datapoints for treatment and control
    number_non_null_treatments = (~treatments.isnull()).sum(axis=1)
    number_non_null_controls = (~controls.isnull()).sum(axis=1)
    
    # First check if our estimate is based on a single datapoint
    # This is true if treatments have only one non-null value
                                                            
    based_on_single_datapoint = number_non_null_treatments == 1
    # Or the controls have only one non-null value
    based_on_single_datapoint |= number_non_null_controls == 1
    
    # Now let's see if we need to impute some of the estimates
    
    
    # Start with copying the estimates
    imputed_estimate = estimate.copy()
    
    # Find datapoints w/o estimates
    no_estimate = estimate.isnull()
    
    # Impute positive infinities
    imputed_estimate.loc[
        # Where we have no estimate, but treatment data and no control data
        no_estimate & (number_non_null_treatments > 0)  & (number_non_null_controls == 0),
    ] = np.inf
    
    # Impute negative infinities
    imputed_estimate.loc[
        # Where we have no estimate, no treatment data but some control data
        no_estimate & (number_non_null_treatments == 0)  & (number_non_null_controls > 0),
    ] = -np.inf
    
    # Impute estimates where we have some treatment and control data, but still no estimate
    # (this happens when only one of the controls give a value, for instance)
    imputable_entries = no_estimate & (number_non_null_treatments > 0)  & (number_non_null_controls > 0)
    
    # Here we are using the results from the "sanity checks above" - that the mean estimates are
    # the same as model estimates..
    imputed_estimate.loc[
        imputable_entries,
    ] = treatments.loc[imputable_entries].mean(axis=1) - controls.loc[imputable_entries].mean(axis=1)
    
    
    # We can get the list of imputed estimates by comparing the null estimates between the imputed and (real) column:
    
    is_imputed = (~imputed_estimate.isnull()) & (no_estimate)
    
    # assert that for all non-null estimates imputed estimate is the same (i.e. that we didn't override)
    assert_array_equal(imputed_estimate.loc[~no_estimate], estimate.loc[~no_estimate])
    
    
    return pd.DataFrame({
        'logFC_imputed': imputed_estimate,
        'logFC_is_imputed': is_imputed,
        'logFC_based_on_single_datapoint': based_on_single_datapoint, 
    })
    


In [ ]:
results_flags_and_imputations = {}


results_flags_and_imputations['H3K4me1vsControl'] = flag_and_or_impute(
    results['H3K4me1vsControl', 'logFC'],
    data_to_model_by_experiment['H3K4me1'],
    data_to_model_by_experiment['Controls']
)

results_flags_and_imputations['H3K4me3vsControl'] = flag_and_or_impute(
    results['H3K4me3vsControl', 'logFC'],
    data_to_model_by_experiment['H3K4me3'],
    data_to_model_by_experiment['Controls']
)

results_flags_and_imputations['H3K4me3vsH3K4me1'] = flag_and_or_impute(
    results['H3K4me3vsH3K4me1', 'logFC'],
    data_to_model_by_experiment['H3K4me3'],
    data_to_model_by_experiment['H3K4me1']
)

results_flags_and_imputations = pd.concat(results_flags_and_imputations, axis=1)
results_flags_and_imputations

In [ ]:
results_flags_and_imputations['H3K4me1vsControl', 'logFC_is_imputed'].value_counts()

In [ ]:
results_flags_and_imputations['H3K4me3vsControl', 'logFC_is_imputed'].value_counts()

In [ ]:
results_flags_and_imputations['H3K4me3vsH3K4me1', 'logFC_is_imputed'].value_counts()

We should augment the data_comment columns

In [ ]:
def comment_string(flags, column):
    
    if flags['logFC_is_imputed'] and flags['logFC_based_on_single_datapoint']:
        return f'logFC({column}) estimation failed and was imputed based on a single datapoint only'
    elif flags['logFC_is_imputed']:
        return f'logFC({column}) estimation failed and was imputed'
    elif flags['logFC_based_on_single_datapoint']:
        return f'logFC({column}) estimation was based on a single datapoint only'
    elif pd.isnull(flags['logFC_imputed']):
        return f'logFC({column}) estimation failed'
    

In [ ]:
comments_results_flags = {
    col: results_flags_and_imputations[col].apply(comment_string, column=col, axis=1) for col in contrasts_matrix_as_df.columns
}

In [ ]:
comments_results_flags['H3K4me3vsH3K4me1'].value_counts()

In [ ]:
comments_results_flags['H3K4me1vsControl'].value_counts()

In [ ]:
comments_results_flags['H3K4me3vsControl'].value_counts()

Let's check the imputation results for the representative null proteins below, a reminder of what the data to model for these proteins looks like:

In [ ]:
data_to_model.loc[REPRESENTATIVE_NULLS]

The corresponding mean estimates for the coefficients minus simple mean of controls (for comparison) where approrpiate

In [ ]:
data_to_model_by_experiment['H3K4me1'].loc[REPRESENTATIVE_NULLS].mean(axis=1) - data_to_model_by_experiment['Controls'].loc[REPRESENTATIVE_NULLS].mean(axis=1)

In [ ]:
data_to_model_by_experiment['H3K4me3'].loc[REPRESENTATIVE_NULLS].mean(axis=1) - data_to_model_by_experiment['Controls'].loc[REPRESENTATIVE_NULLS].mean(axis=1)

The imputed estimates are shown below:

In [ ]:
results_flags_and_imputations.loc[REPRESENTATIVE_NULLS].loc(axis=1)[:, 'logFC_imputed']

Please verify that they are intuitively correct:

1. `PHF8` should have infinite enrichments for `H3K4me1vsControl` and `H3K4me3vsControl`, as it has no estimate for `ExperimentH3` and `ExperimentH4` coefficients (as _all_ values in the two controls are missing), but is detected in `H3K4me3` and `H3K4me1` experiments. 
2. `FAM98A`, `EIF4G1` and `ZMYND11` should have values imputed with fold change against simple mean of controls as they have estimates for one control (H3 or H4) but not both. 
3. `H3K4me1vsControl` should be -inf for `PSMC1` and `TUBB2A` , while `H3K4me3vsH3K4me1` should be +inf for these proteins as they have no estimate for `ExperimentH3K4me1`. Similarly impute `SNTB2` to have `H3K4me3vsControl` and `H3K4me3vsH3K4me1`  estimates of negative infinity as this protein has no estimate for `ExperimentH3K4me3`.
4. Finally, since `TNN3` has estimates only for the `Control`-associated samples, it should have `H3K4me1vsControl` and `H3K4me3vsControl` imputed to infinity, but no estimate for `H3K4me3vsH3K4me1`.

### Heatmaps of imputed proteins

#### H3K4me3vsControl

In [ ]:
_colname = 'H3K4me3vsControl'
_imputed = results_flags_and_imputations.loc[results_flags_and_imputations[_colname, 'logFC_is_imputed'], _colname].sort_values(by='logFC_imputed', ascending=False)
_imputed

In [ ]:
from matplotlib.gridspec import GridSpec
figure = plt.figure(figsize=(20*FIVE_MM_IN_INCH, 28*FIVE_MM_IN_INCH), constrained_layout=True)

gs = GridSpec(2, 2, figure=figure, width_ratios=(1, 12), height_ratios=(10,1))
ax_legends = figure.add_subplot(gs[1,:])
ax_legends.axis('off')
cax_left = ax_legends.inset_axes([0.02, 0.58, 0.2, 0.2], transform=ax_legends.transAxes)
cax_right = ax_legends.inset_axes([0.32, 0.58, 0.2, 0.2], transform=ax_legends.transAxes)

cax_left.set_title("Imputed logFC estimate")
cax_right.set_title("Normalised data")

ax_left = figure.add_subplot(gs[0, 0])
ax_right = figure.add_subplot(gs[0, 1], sharey=ax_left)


vmax = np.ceil(_imputed['logFC_imputed'].abs().replace(np.inf, np.nan).dropna().max())
vmin = -vmax

heatmap_right = sns.heatmap(
    data_to_model.loc[_imputed.index], 
    cmap='viridis', robust=True, 
    ax=ax_right, yticklabels=1, 
    cbar=True,
    cbar_ax = cax_right,
    cbar_kws=dict(orientation='horizontal'),
    linewidth=0.1,
    linecolor='black',
)
heatmap_left = sns.heatmap(
    _imputed[['logFC_imputed']].replace(-np.inf, vmin -1).replace(np.inf, vmax + 1), 
    cmap='RdBu_r', 
    center=0, 
    vmin=vmin, vmax=vmax,
    ax=ax_left, yticklabels=1, 
    cbar=True,
    cbar_ax=cax_left,
    cbar_kws=dict(orientation='horizontal'),
    linewidth=0.1,
    linecolor='black',
)

heatmap_right.xaxis.tick_top()
heatmap_right.xaxis.set_tick_params(length=0, rotation=90)

heatmap_left.xaxis.tick_top()
heatmap_left.xaxis.set_tick_params(length=0, rotation=90)

heatmap_right.yaxis.set_tick_params(length=0)
heatmap_left.yaxis.set_tick_params(length=0)

for ax in [heatmap_left, heatmap_right]:
    for spine in ax.spines:
        ax.spines[spine].set_visible(True)

for tick in heatmap_right.yaxis.get_ticklabels():
    tick.set_visible(False)

heatmap_right.set_title(f"Proteins with imputed logFC estimates for {_colname}")

_fname = OUTPUT_DIRECTORY / f'06-modelling-imputed-values-for-{_colname}.pdf'
_caption = f"""

Heatmap of the imputed `logFC` estimates for the {_colname} coefficient.
The proteins are sorted by the estimate, descending.

The left heatmap plots the imputed `logFC` estimate, the colour axis limits are set to [{vmin}, {vmax}].
Note that some imputed logFC estimates are infinite (which may be both positive or negative). Such points are displayed at the darkest shade of respective colour.

The right heatmap shows the data used for modelling in viridis colour scale. 
Note that all imptued estimates were infered from partial data only.
"""
plt.savefig(_fname, bbox_inches='tight', dpi=DPI)
with open(str(_fname) + '.caption.md', 'w') as f:
    f.write(_caption)
    print(_caption)


#### H3K4me1vsControl

In [ ]:
_colname = 'H3K4me1vsControl'
_imputed = results_flags_and_imputations.loc[results_flags_and_imputations[_colname, 'logFC_is_imputed'], _colname].sort_values(by='logFC_imputed', ascending=False)
_imputed

In [ ]:
from matplotlib.gridspec import GridSpec
figure = plt.figure(figsize=(20*FIVE_MM_IN_INCH, 28*FIVE_MM_IN_INCH), constrained_layout=True)

gs = GridSpec(2, 2, figure=figure, width_ratios=(1, 12), height_ratios=(10,1))
ax_legends = figure.add_subplot(gs[1,:])
ax_legends.axis('off')
cax_left = ax_legends.inset_axes([0.02, 0.58, 0.2, 0.2], transform=ax_legends.transAxes)
cax_right = ax_legends.inset_axes([0.32, 0.58, 0.2, 0.2], transform=ax_legends.transAxes)

cax_left.set_title("Imputed logFC estimate")
cax_right.set_title("Normalised data")

ax_left = figure.add_subplot(gs[0, 0])
ax_right = figure.add_subplot(gs[0, 1], sharey=ax_left)


vmax = np.ceil(_imputed['logFC_imputed'].abs().replace(np.inf, np.nan).dropna().max())
vmin = -vmax

heatmap_right = sns.heatmap(
    data_to_model.loc[_imputed.index], 
    cmap='viridis', robust=True, 
    ax=ax_right, yticklabels=1, 
    cbar=True,
    cbar_ax = cax_right,
    cbar_kws=dict(orientation='horizontal'),
    linewidth=0.1,
    linecolor='black',
)
heatmap_left = sns.heatmap(
    _imputed[['logFC_imputed']].replace(-np.inf, vmin -1).replace(np.inf, vmax + 1), 
    cmap='RdBu_r', 
    center=0, 
    vmin=vmin, vmax=vmax,
    ax=ax_left, yticklabels=1, 
    cbar=True,
    cbar_ax=cax_left,
    cbar_kws=dict(orientation='horizontal'),
    linewidth=0.1,
    linecolor='black',
)

heatmap_right.xaxis.tick_top()
heatmap_right.xaxis.set_tick_params(length=0, rotation=90)

heatmap_left.xaxis.tick_top()
heatmap_left.xaxis.set_tick_params(length=0, rotation=90)

heatmap_right.yaxis.set_tick_params(length=0)
heatmap_left.yaxis.set_tick_params(length=0)

for ax in [heatmap_left, heatmap_right]:
    for spine in ax.spines:
        ax.spines[spine].set_visible(True)

for tick in heatmap_right.yaxis.get_ticklabels():
    tick.set_visible(False)

heatmap_right.set_title(f"Proteins with imputed logFC estimates for {_colname}")

_fname = OUTPUT_DIRECTORY / f'06-modelling-imputed-values-for-{_colname}.pdf'
_caption = f"""

Heatmap of the imputed `logFC` estimates for the {_colname} coefficient.
The proteins are sorted by the estimate, descending.

The left heatmap plots the imputed `logFC` estimate, the colour axis limits are set to [{vmin}, {vmax}].
Note that some imputed logFC estimates are infinite (which may be both positive or negative). Such points are displayed at the darkest shade of respective colour.

The right heatmap shows the data used for modelling in viridis colour scale. 
Note that all imptued estimates were infered from partial data only.
"""
plt.savefig(_fname, bbox_inches='tight', dpi=DPI)
with open(str(_fname) + '.caption.md', 'w') as f:
    f.write(_caption)
    print(_caption)


#### H3K4me3vsH3K4me1

In [ ]:
_colname = 'H3K4me3vsH3K4me1'
_imputed = results_flags_and_imputations.loc[results_flags_and_imputations[_colname, 'logFC_is_imputed'], _colname].sort_values(by='logFC_imputed', ascending=False)
_imputed

In [ ]:
from matplotlib.gridspec import GridSpec
figure = plt.figure(figsize=(20*FIVE_MM_IN_INCH, 28*FIVE_MM_IN_INCH), constrained_layout=True)

gs = GridSpec(2, 2, figure=figure, width_ratios=(1, 12), height_ratios=(10,1))
ax_legends = figure.add_subplot(gs[1,:])
ax_legends.axis('off')
cax_left = ax_legends.inset_axes([0.02, 0.58, 0.2, 0.2], transform=ax_legends.transAxes)
cax_right = ax_legends.inset_axes([0.32, 0.58, 0.2, 0.2], transform=ax_legends.transAxes)

cax_left.set_title("Imputed logFC estimate")
cax_right.set_title("Normalised data")

ax_left = figure.add_subplot(gs[0, 0])
ax_right = figure.add_subplot(gs[0, 1], sharey=ax_left)


vmax = np.ceil(_imputed['logFC_imputed'].abs().replace(np.inf, np.nan).dropna().max())
if pd.isnull(vmax):
    vmax = 1.0

vmin = -vmax

heatmap_right = sns.heatmap(
    data_to_model.loc[_imputed.index], 
    cmap='viridis', robust=True, 
    ax=ax_right, yticklabels=1, 
    cbar=True,
    cbar_ax = cax_right,
    cbar_kws=dict(orientation='horizontal'),
    linewidth=0.1,
    linecolor='black',
)
heatmap_left = sns.heatmap(
    _imputed[['logFC_imputed']].replace(-np.inf, vmin -1).replace(np.inf, vmax + 1), 
    cmap='RdBu_r', 
    center=0, 
    vmin=vmin, vmax=vmax,
    ax=ax_left, yticklabels=1, 
    cbar=True,
    cbar_ax=cax_left,
    cbar_kws=dict(orientation='horizontal'),
    linewidth=0.1,
    linecolor='black',
)

heatmap_right.xaxis.tick_top()
heatmap_right.xaxis.set_tick_params(length=0, rotation=90)

heatmap_left.xaxis.tick_top()
heatmap_left.xaxis.set_tick_params(length=0, rotation=90)

heatmap_right.yaxis.set_tick_params(length=0)
heatmap_left.yaxis.set_tick_params(length=0)

for ax in [heatmap_left, heatmap_right]:
    for spine in ax.spines:
        ax.spines[spine].set_visible(True)

for tick in heatmap_right.yaxis.get_ticklabels():
    tick.set_visible(False)

heatmap_right.set_title(f"Proteins with imputed logFC estimates for {_colname}")

_fname = OUTPUT_DIRECTORY / f'06-modelling-imputed-values-for-{_colname}.pdf'
_caption = f"""

Heatmap of the imputed `logFC` estimates for the {_colname} coefficient.
The proteins are sorted by the estimate, descending.

The left heatmap plots the imputed `logFC` estimate, the colour axis limits are set to [{vmin}, {vmax}].
Note that some imputed logFC estimates are infinite (which may be both positive or negative). Such points are displayed at the darkest shade of respective colour.

The right heatmap shows the data used for modelling in viridis colour scale. 
Note that all imptued estimates were infered from partial data only.
"""
plt.savefig(_fname, bbox_inches='tight', dpi=DPI)
with open(str(_fname) + '.caption.md', 'w') as f:
    f.write(_caption)
    print(_caption)


## Output

At this point this notebook is finished, so it's time to collect the results and visualise


Let's start with a combined "comment column",
which joins the filtering comment and the flag comments

In [ ]:
full_comments = pd.DataFrame({'filter_comment': data_comment, 'norm_comment': comments_norm, **comments_results_flags})
full_comments

Use a separator of ';'

In [ ]:
full_comments = full_comments.apply(lambda x: '; '.join(x.dropna()), axis=1)
full_comments

In [ ]:
full_comments.value_counts()

Now gather all the data that we would need to save:

In [ ]:
_comments = pd.DataFrame(full_comments.copy())
_comments.columns = pd.MultiIndex.from_tuples([('comment', 'comment')])

_coef_estimates = coef_estimates_design.copy()
_coef_estimates.columns = pd.MultiIndex.from_tuples([('coefficient_estimates', c) for c in _coef_estimates.columns])

_norm_data = data_numeric_log2_normalised.copy()
_norm_data.columns = pd.MultiIndex.from_tuples([('normalised_data', c) for c in _norm_data.columns])

In [ ]:
full_results = _norm_data.join(_comments).join(results).join(_coef_estimates).join(results_flags_and_imputations)
full_results

For storage, remove multi-index:

In [ ]:
full_results_no_multiindex = full_results.copy()
full_results_no_multiindex.columns = ['__'.join(c) for c in full_results.columns]

In [ ]:
full_results_no_multiindex.to_csv(OUTPUT_DIRECTORY / '07-output-full_results.csv')

## Visualisation

Finally, plot the key results

### Venn diagrams

In [ ]:
from matplotlib_venn import venn3_unweighted

In [ ]:
ax = plt.gca()

set_labels = coef_estimates_contrasts.columns
set_members = [ set(full_results[full_results[c, 'significant'].fillna(False)].index) for c in set_labels ]

venn3_unweighted(set_members, set_labels=set_labels)

ax.set_title("Venn diagram of proteins of significantly non-zero fold changes")


_fname = OUTPUT_DIRECTORY / f'08-venn-diagram-significant.pdf'
_caption = f"""

Venn diagram of proteins whose fold change estimates are statistically non zero. (limma, at FDR {FDR_THRESHOLD}, BH correction).
Note that the venn diagram areas are not representative of the counts in the slices.
"""
plt.savefig(_fname, bbox_inches='tight', dpi=DPI)
with open(str(_fname) + '.caption.md', 'w') as f:
    f.write(_caption)
    print(_caption)

In [ ]:
ax = plt.gca()

set_labels = coef_estimates_contrasts.columns


set_members = [ 
     set(full_results[full_results[c, 'significant'].fillna(False)].index) - set(full_results[full_results[c, 'logFC_based_on_single_datapoint'].fillna(False)].index)
    for c in coef_estimates_contrasts.columns
]
venn3_unweighted(set_members, set_labels=set_labels)

ax.set_title("Venn diagram of proteins of significantly non-zero fold changes\nthat are based on more than one data point")


_fname = OUTPUT_DIRECTORY / f'08-venn-diagram-significant-more-than-one-datapoint.pdf'
_caption = f"""

Venn diagram of proteins whose fold change estimates are statistically non zero. (limma, at FDR {FDR_THRESHOLD}, BH correction).
Note that the venn diagram areas are not representative of the counts in the slices.
"""
plt.savefig(_fname, bbox_inches='tight', dpi=DPI)
with open(str(_fname) + '.caption.md', 'w') as f:
    f.write(_caption)
    print(_caption)

In [ ]:
ax = plt.gca()

set_labels = coef_estimates_contrasts.columns


set_members = [ 
     set(full_results[full_results[c, 'significant'].fillna(False)].index) - set(full_results[full_results[c, 'logFC_based_on_single_datapoint'].fillna(False)].index)
    for c in coef_estimates_contrasts.columns
]
venn3_unweighted(set_members, set_labels=set_labels)

ax.set_title("Venn diagram of proteins of significantly non-zero fold changes\nthat are based on more than one data point")


_fname = OUTPUT_DIRECTORY / f'08-venn-diagram-significant-more-than-one-datapoint.pdf'
_caption = f"""

Venn diagram of proteins whose fold change estimates are statistically non zero. (limma, at FDR {FDR_THRESHOLD}, BH correction),
and whose estimates are based on >1 data point.
Note that the venn diagram areas are not representative of the counts in the slices.
"""
plt.savefig(_fname, bbox_inches='tight', dpi=DPI)
with open(str(_fname) + '.caption.md', 'w') as f:
    f.write(_caption)
    print(_caption)

In [ ]:
ax = plt.gca()

set_labels = coef_estimates_contrasts.columns


set_members = [ 
    set(full_results[full_results[c, 'significant'].fillna(False)].index) 
        & set(full_results[(full_results['H3K4me1vsControl', 'logFC_imputed'] > 0).fillna(False)].index)
        & set(full_results[(full_results['H3K4me3vsControl', 'logFC_imputed'] > 0).fillna(False)].index)
    for c in coef_estimates_contrasts.columns
]
venn3_unweighted(set_members, set_labels=set_labels)

ax.set_title("Venn diagram of proteins of significantly non-zero fold changes\ngiven that me1/me3 estimates are positive")


_fname = OUTPUT_DIRECTORY / f'08-venn-diagram-significant-me1me3-positive.pdf'
_caption = f"""

Venn diagram of proteins whose fold change estimates are statistically non zero. (limma, at FDR {FDR_THRESHOLD}, BH correction).
And whose H3K4me1vsControl and H3K4me3vsControl imputed logFC estimates are greater than zero
"""
plt.savefig(_fname, bbox_inches='tight', dpi=DPI)
with open(str(_fname) + '.caption.md', 'w') as f:
    f.write(_caption)
    print(_caption)

### Scatterplots

In [ ]:
from sklearn.ensemble import IsolationForest
from adjustText import adjust_text

fig = plt.figure(figsize=(50*FIVE_MM_IN_INCH, 10*FIVE_MM_IN_INCH), constrained_layout=True)

axes = fig.subplots(nrows=1, ncols=5, sharex=True, sharey=True)

_x = ('H3K4me3vsControl', 'logFC')
_y = ('H3K4me1vsControl', 'logFC')
_df = full_results.dropna(subset=[_x, _y])
_approx_nlabels = 20

for ax, col, color in zip(axes, [None] + ['normalisation_proteins'] + list(contrasts_matrix_as_df.columns), sns.color_palette('Set1')):

    sns.histplot(
        x=_x, 
        y=_y, 
        data=_df,
        cmap=sns.blend_palette(['#000000','#969696'], as_cmap=True),
        ax=ax,
    )
    
    if col is None:
        if_ = IsolationForest(random_state=42, contamination=np.clip(_approx_nlabels/len(_df), 1e-5, 0.5))
        
        outliers = if_.fit_predict(_df[[_x, _y]])
        outliers = pd.Series((outliers == -1), index=_df.index)
        
        texts = []
        for ix in outliers[outliers].index:
            texts.append(ax.text(_df.loc[ix, _x], _df.loc[ix, _y], ix, color='black'))
        
        adjust_text(texts, arrowprops=dict(arrowstyle='-'), ax=ax)
    elif col == 'normalisation_proteins':
        
        
        ax.scatter(
            _df.loc[HISTONES, _x],
            _df.loc[HISTONES, _y],
            color=color,
            edgecolor='black',
            alpha=0.8,
            s=5,
            label=col,
        )
        
        texts = []
        for ix in HISTONES:
            texts.append(ax.text(_df.loc[ix, _x], _df.loc[ix, _y], ix, color=color))
        
        adjust_text(texts, arrowprops=dict(arrowstyle='-'), ax=ax)
    
    else:
        subdf = _df[_df[col, 'significant'].fillna(False)]
    
        if_ = IsolationForest(random_state=42, contamination=np.clip(_approx_nlabels/len(subdf), 1e-5, 0.5))
        outliers = if_.fit_predict(subdf[[_x, _y]])
        outliers = pd.Series((outliers == -1), index=subdf.index)
        
        ax.scatter(
            subdf[_x],
            subdf[_y],
            color=color,
            edgecolor='black',
            alpha=0.8,
            s=5,
            label=col,
        )
        
        ax.legend(title="Significant")
        
        texts = []
        for ix in outliers[outliers].index:
            texts.append(ax.text(_df.loc[ix, _x], _df.loc[ix, _y], ix, color=color))
        
        adjust_text(texts, arrowprops=dict(arrowstyle='-'), ax=ax)

    ax.grid(False)
    
    ax.axvline(0, color='black', linestyle='-', linewidth=0.5)
    ax.axhline(0, color='black', linestyle='-', linewidth=0.5)
    
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    min_ = np.min([xlim, ylim])
    max_ = np.max([xlim, ylim])
    
    ax.plot([min_, max_], [min_, max_], linestyle=':', color='k')
    ax.set_xlim(*xlim, *ylim)
    ax.set_xlabel('log2FC(H3K4me3/Controls)')
    ax.set_ylabel('log2FC(H3K4me1/Controls)')
   
    sns.despine(offset=5, ax=ax)
    
_fname = OUTPUT_DIRECTORY / f'09-scatterplot-full-results_logfc.pdf'
_caption = f"""
A scatterplot diagram of model estimates.
Log2 fold changes me3 vs controls (H3 and H4) of all proteins are plotted on the x axis, and the log2FC(me1/controls) on the y.

The first plot simply plots the whole data, the second plot highlights proteins used to normalise the data, while the remaining three plots highlight proteins for which
the corresponding estimates are statistically non zero. (limma, at FDR {FDR_THRESHOLD}, BH correction).
Labels are displayed for a handful of outlier proteins.
"""
plt.savefig(_fname, bbox_inches='tight', dpi=DPI)
with open(str(_fname) + '.caption.md', 'w') as f:
    f.write(_caption)
    print(_caption)

And a zoomed in version of this plot

In [ ]:
from sklearn.ensemble import IsolationForest
from adjustText import adjust_text

fig = plt.figure(figsize=(50*FIVE_MM_IN_INCH, 10*FIVE_MM_IN_INCH), constrained_layout=True)

axes = fig.subplots(nrows=1, ncols=5, sharex=True, sharey=True)

_x = ('H3K4me3vsControl', 'logFC')
_y = ('H3K4me1vsControl', 'logFC')

_limits = [-3, 5]

_df = full_results.dropna(subset=[_x, _y])
_mask = _df[_x].between(*_limits) & _df[_y].between(*_limits)
to_drop = _df[~_mask]
drop_statement = '{:,} proteins outside of axis limits: {}'.format(
    len(to_drop), ['{} (x={:.2f}, y={:.2f})'.format(ix, row[_x], row[_y]) for ix, row in to_drop.sort_index().iterrows()]
)

_df = _df[_mask]

_approx_nlabels = 20

for ax, col, color in zip(axes, [None] + ['normalisation_proteins'] + list(contrasts_matrix_as_df.columns), sns.color_palette('Set1')):

    sns.histplot(
        x=_x, 
        y=_y, 
        data=_df,
        cmap=sns.blend_palette(['#000000','#969696'], as_cmap=True),
        ax=ax,
    )
    
    if col is None:
        if_ = IsolationForest(random_state=42, contamination=np.clip(_approx_nlabels/len(_df), 1e-5, 0.5))
        
        outliers = if_.fit_predict(_df[[_x, _y]])
        outliers = pd.Series((outliers == -1), index=_df.index)
        
        texts = []
        for ix in outliers[outliers].index:
            texts.append(ax.text(_df.loc[ix, _x], _df.loc[ix, _y], ix, color='black'))
        
        adjust_text(texts, arrowprops=dict(arrowstyle='-'), ax=ax)
    elif col == 'normalisation_proteins':
        
        
        ax.scatter(
            _df.loc[HISTONES, _x],
            _df.loc[HISTONES, _y],
            color=color,
            edgecolor='black',
            alpha=0.8,
            s=5,
            label=col,
        )
        
        texts = []
        for ix in HISTONES:
            texts.append(ax.text(_df.loc[ix, _x], _df.loc[ix, _y], ix, color=color))
        
        adjust_text(texts, arrowprops=dict(arrowstyle='-'), ax=ax)
    
    else:
        subdf = _df[_df[col, 'significant'].fillna(False)]
    
        if_ = IsolationForest(random_state=42, contamination=np.clip(_approx_nlabels/len(subdf), 1e-5, 0.5))
        outliers = if_.fit_predict(subdf[[_x, _y]])
        outliers = pd.Series((outliers == -1), index=subdf.index)
        
        ax.scatter(
            subdf[_x],
            subdf[_y],
            color=color,
            edgecolor='black',
            alpha=0.8,
            s=5,
            label=col,
        )
        
        ax.legend(title="Significant")
        ax.set_xlim(*_limits)
        ax.set_ylim(*_limits)
        texts = []
        for ix in outliers[outliers].index:
            texts.append(ax.text(_df.loc[ix, _x], _df.loc[ix, _y], ix, color=color))
        
        adjust_text(texts, arrowprops=dict(arrowstyle='-'), ax=ax)

    ax.grid(False)
    
    ax.axvline(0, color='black', linestyle='-', linewidth=0.5)
    ax.axhline(0, color='black', linestyle='-', linewidth=0.5)
    
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    min_ = np.min([xlim, ylim])
    max_ = np.max([xlim, ylim])
    
    ax.plot([min_, max_], [min_, max_], linestyle=':', color='k')
    ax.set_xlim(*xlim, *ylim)
    ax.set_xlabel('log2FC(H3K4me3/Controls)')
    ax.set_ylabel('log2FC(H3K4me1/Controls)')
   
    sns.despine(offset=5, ax=ax)
    
_fname = OUTPUT_DIRECTORY / f'09-scatterplot-full-results_logfc_zoomed_in.pdf'
_caption = f"""
A scatterplot diagram of model estimates.
Log2 fold changes me3 vs controls (H3 and H4) of all proteins are plotted on the x axis, and the log2FC(me1/controls) on the y.

The first plot simply plots the whole data, the second plot highlights proteins used to normalise the data, while the remaining three plots highlight proteins for which
the corresponding estimates are statistically non zero. (limma, at FDR {FDR_THRESHOLD}, BH correction).
Labels are displayed for a handful of outlier proteins.

Note that this plot zooms in to the area defined by the following limits: {_limits}.
{drop_statement}
"""
plt.savefig(_fname, bbox_inches='tight', dpi=DPI)
with open(str(_fname) + '.caption.md', 'w') as f:
    f.write(_caption)
    print(_caption)